<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Marcelo Rojas
- Nombre de alumno 2: Sebastián Sanhueza


### **Link de repositorio de GitHub:** `https://github.com/Marcelorojasb/MDS7202`

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
#!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito.
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos]

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [4]:
def custom_features(dataframe_in):
    """Función que calcula las características LRMFP.

    Parmeters
    ----------
    dataframe_in: pd.DataFrame
        Dataframe con los datos de retail.

    Return
    -------
    customer_lrmfp: pd.DataFrame
        Dataframe con las características LRMFP.
    """

    # Copiamos los datos de entrada
    df_copy = dataframe_in.copy()

    # Agrupamos las transacciones de acuerdo al ID de los clientes
    retail_grouped = df_copy.groupby('Customer ID')

    # Primero se calculan y registran la fecha más reciente y más antigua en la cual un cliente haya realizado una transacción
    customer_dates = retail_grouped['InvoiceDate'].agg(['min', 'max'])


    #------------------------------------------------------------------------------------------------------------------------------------
    # Calculamos Length(L) para cada cliente
    customer_length = (customer_dates['max'] - customer_dates['min']).dt.days

    # Calculamos Recency(R) para cada cliente
    most_recent_date = df_copy['InvoiceDate'].max() + datetime.timedelta(days=1) # Obtenemos la fecha más reciente dentro del Dataframe
    customer_recency = (most_recent_date - customer_dates['max']).dt.days # Obtenemos hace cuantos días el cliente realizó la última compra

    # Calculamos Monetary(M) para cada cliente
    customer_monetary = retail_grouped.apply(lambda x: ((x['Quantity'] * x['Price']).sum() / x['Invoice'].count()).round(2))

    # Calculamos Frequency(F) para cada cliente
    customer_frequency = retail_grouped['Invoice'].count()

    # Calculamos Periodicity(P) para cada cliente
    diff_date = retail_grouped['InvoiceDate'].diff().dt.days # Calculamos los días de diferencia entre cada visita para cada cliente
    # La función diff() calcula la diferencia entre visitas para cada cliente, pero entrega una serie desagrupada, por lo que se tienen que asignar los
    # ID a cada diferencia y agrupar nuevamente para calcular la desviación estándar
    customer_diff_date = pd.concat([df_copy['Customer ID'], diff_date],axis=1)
    customer_periodicity = customer_diff_date.groupby('Customer ID').agg('std').fillna(0.0).round(1) # Definimos que los datos NaN se vuelvan cero
    #------------------------------------------------------------------------------------------------------------------------------------


    # Concatenamos cada Dataframe con las características de acuerdo a cada cliente
    customer_lrmfp = pd.concat([customer_length, customer_recency, customer_frequency, customer_monetary, customer_periodicity], axis=1)

    # Restablecemos el índice del DataFrame resultante y le asignamos los nombres de las columnas
    customer_lrmfp.reset_index(inplace=True)
    customer_lrmfp.columns = ['Customer ID', 'Length', 'Recency', 'Frequency', 'Monetary','Periodicity']

    return customer_lrmfp

**Respuesta de Pauta:**

In [5]:
# Evaluamos la función con el Dataframe entregado
features=custom_features(df_retail)
features

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,196,165,33,11.30,21.7
1,12347.0,37,3,71,18.64,4.4
2,12348.0,0,74,20,11.11,0.0
3,12349.0,181,43,102,26.19,16.2
4,12351.0,0,11,21,14.33,0.0
...,...,...,...,...,...,...
4309,18283.0,275,18,217,2.85,11.8
4310,18284.0,0,67,28,16.49,0.0
4311,18285.0,0,296,12,35.58,0.0
4312,18286.0,247,112,67,19.35,30.4


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [6]:
class MinMax(BaseEstimator, TransformerMixin):
    """Clase que escala y transforma cada columna de un Dataframe tal que
       queden en un rango [0, 1]."""
    def fit(self, X, y=None):
        """Función que calcula el mínimo y el máximo para ser usados en la
           transformación de los datos.

        Parameters
        ------------------------------------------------------
        X: pandas.Dataframe
           Datos usados para calcular el mínimo y el máximo de cada característica
           que se utilizaran para escalar.

        y: None
           Ignorado por compatibilidad.

        Returns
        ------------------------------------------------------
        self: MinMax
              Escalador entrenado."""
        self.min_ = np.nanmin(X, axis=0) # Extracción del mínimo de cada característica
        self.max_ = np.nanmax(X, axis=0) # Extracción del máximo de cada característica
        self.denominator_ = self.max_ - self.min_ # Cálculo del denominador en la ecuación del escalador
        return self

    def transform(self, X):
        """Función que escala las columnas de X.

        Parameters
        ------------------------------------------------------
        X: pandas.Dataframe
           Datos a transformar.

        Returns
        ------------------------------------------------------
        X_transform: pandas.Dataframe
            Datos transformados."""
        X_transform = (X - self.min_)/self.denominator_ # Se aplica la ecuación del escalador a los datos
        return X_transform


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo.

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior.
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [7]:
pipe = Pipeline([
    ('custom_features', FunctionTransformer(custom_features)), # Primer paso, obtención de las características LRMFP
    ('scaler', ColumnTransformer(transformers=[
        ('num', MinMax(), ['Length', 'Recency', 'Frequency', 'Monetary','Periodicity']) # Segundo paso, escalamos todas las columnas que no son categóricas
    ])),
    ('reducer', TSNE(n_components=2, random_state=0, learning_rate='auto', init='random')) # Tercer paso, obtener las 2 componentes más relevantes
])

X_reduced = pd.DataFrame(pipe.fit_transform(df_retail), columns=['x', 'y'])
X_reduced

,x,y
0,-4.819564,-25.037743
1,-11.395205,15.289492
2,-57.855400,28.058701
3,20.214483,-0.972048
4,-9.449186,-47.144745
...,...,...
4309,47.670322,-15.267982
4310,-47.603809,39.134460
4311,-41.404003,16.725662
4312,13.717017,-29.605194


A continuación visualizamos las 2 componentes principales obtenidas:

In [8]:
fig = px.scatter(X_reduced, x="x", y="y")
fig.update_layout(title="Espacio de dimensión reducida con T-SNE")

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [9]:
pipe = Pipeline([
    ('custom_features', FunctionTransformer(custom_features)), # Primer paso, obtención de las características LRMFP
    ('scaler', ColumnTransformer(transformers=[
        ('num', MinMax(), ['Length', 'Recency', 'Frequency', 'Monetary','Periodicity']) # Segundo paso, escalamos todas las columnas que no son categóricas
    ])),
])

scaled_features = pd.DataFrame(pipe.fit_transform(df_retail))


intertias = [
    [i, KMeans(n_clusters=i, random_state=0, n_init=10).fit(scaled_features).inertia_]
    for i in range(1, 20)
]

intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(10)

,n° clusters,inertia
0,1,879.467346
1,2,355.284331
2,3,192.335787
3,4,131.620248
4,5,109.482046
5,6,93.468502
6,7,78.718301
7,8,69.216253
8,9,63.030907
9,10,57.421484


In [10]:
 px.line(
    intertias,
    x="n° clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
)

Vemos que la curva efectivamente tiene forma de "codo". Podemos ver que el codo de la curva se encuentra cercano a k=5. Por lo que el número óptimo de clúster es aproximadamente 5.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [16]:
pipe = Pipeline([
    ('custom_features', FunctionTransformer(custom_features)), # Primer paso, obtención de las características LRMFP
    ('scaler', ColumnTransformer(transformers=[
        ('num', MinMax(), ['Length', 'Recency', 'Frequency', 'Monetary','Periodicity']) # Segundo paso, escalamos todas las columnas que no son categóricas
    ])),
])
scaled_features = pd.DataFrame(pipe.fit_transform(df_retail))
k_means = KMeans(n_clusters=5, random_state=0, n_init=10).fit(scaled_features)

In [17]:
df_means = features.copy()
df_means['Cluster'] = k_means.labels_
df_means.groupby(['Cluster']).mean(numeric_only=True)

,Length,Recency,Frequency,Monetary,Periodicity
Cluster,,,,,
0,187.412060,61.989950,91.897487,27.213709,20.864221
1,5.701299,305.809524,20.976190,72.104784,1.172078
2,17.822470,43.477702,41.817324,31.877624,2.455746
3,321.560297,23.295918,213.953618,35.915315,20.190724
4,34.698206,187.541599,33.306688,37.986215,5.225938


**Comentarios:**

Es posible observar valores interesantes en algunos de los clústers, lo cual nos permite definir el perfil de clientes de estos.

Vemos que el clúster 1, posee el mayor valor medio para 'Monetary' y también los menores valores medios para 'Periodicity' y 'Frequency'. Esto puede indicar que el perfil de los clientes del clúster 1 corresponden a clientes que no son fieles y compran con baja frecuencia pero que cuando compran, compran por un alto monto de dinero. La no fidelidad de los clientes o baja frecuencia de estos clientes se puede ver también en el valor medio de 'Recency' (es el mayor de todos) lo que indica que estos clientes poseen poco interés para repetir una compra en uno de los locales.

Por otro lado tenemos el clúster 3, el cual posee el mayor valor medio para 'Frequency', el mayor valor medio para 'Length', el menor valor medio para 'Recency' y el segundo mayor valor medio para 'Periodicity'. Como no vemos una diferencia notable para el valor medio de 'Monetary' respecto a otros clústers (a excepción el clúster 1) podemos definir el perfil de estos clientes simplemente como clientes frecuentes y fieles.


Los demás clústers no poseen valores tan extremos como el clúster 1 o 3. Pero si podemos ver que se acercan más unos que a otros clústers. Por ejemplo, el clúster 4 posee valores relativamente cercanos al clúster 1 mientras que el clúster 0 posee valores más cercanos al clúster 3 (incluso tienen casi la misma 'Periodicity' media).




#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [18]:
df_means = X_reduced.copy()
df_means['Cluster'] = k_means.labels_
df_means.groupby(['Cluster']).mean(numeric_only=True)
fig = px.scatter(
        df_means,
        x="x",
        y="y",
        color='Cluster',
        range_x=(df_means["x"].min() - 1, df_means["x"].max() + 1),
        range_y=(df_means["y"].min() - 1, df_means["y"].max() + 1),
    )
fig

Si bien los clúster de 'solapan' entre sí en algunos puntos, en general se pueden observar clúster bien definidos y separados.

Podemos verificar la cercanía de forma visual (anteriormente numérica) entre el clúster 1 y 4 así como la cercanía entre el clúster 3 y 0.

Podemos ver también la separación que existe entre los clústers de clientes poco frecuentes o poco fieles (0 y 3) con los clientes frecuentes o fieles (1 y 4).

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>